In [1]:
from	pyspark.mllib.linalg	import	Vector,	Vectors
from	pyspark.mllib.regression	import	LabeledPoint
import math



In [3]:
def correlation(size, dotProduct, ratingSum,rating2Sum,ratingNormSq,rating2NormSq):
    numerator = size * dotProduct - ratingSum * rating2Sum
    denominator = math.sqrt(size * ratingNormSq - ratingSum * ratingSum)*math.sqrt(size * rating2NormSq - rating2Sum * rating2Sum)
    #if denominator = 0:
     #   corr =0;
    #else:
     #   corr = numerator / denominator
    return  3


def cosineSimilarity(dotProduct,ratingNorm,rating2Norm): 
    if ratingNorm * rating2Norm == 0:
        cosi = 0
    else: 
        cosi = dotProduct / (ratingNorm * rating2Norm)
    
    return cosi



#def jaccardSimilarity(usersInCommon,totalUsers1,totalUsers2): 
 #   union = totalUsers1 + totalUsers2 - usersInCommon
  #  g = usersInCommon / union
  #  return g


In [4]:
Logs = sc.textFile('Data/advertising.csv')
header = Logs.first()
Lorgs_norm = Logs.filter(lambda row: row != header).map(lambda l: l.split(','))
Log2Rec = Lorgs_norm.map(lambda tokens: (int(tokens[2]),int(tokens[1]),1 if tokens[5] == "True" else 0)).cache()

In [24]:
header

u',pub_domain,user_id,viewed,ctr,click'

In [5]:
Log2Rec.take(2)

[(1410160, 1000, 1), (1157972, 1000, 0)]

In [6]:
numRatersPerMovie = Log2Rec.groupBy(lambda (x,y,z): y).map(lambda (x,iterator):(x,len(iterator)))

In [7]:
numRatersPerMovie.take(2)

[(1000, 3), (1002, 11)]

In [8]:
ratingsWithSize = Log2Rec.groupBy(lambda (x,y,z): y).join(numRatersPerMovie).flatMap(lambda(a,(b,c)):((x,c) for x in b))

In [9]:
ratingsWithSize2 = ratingsWithSize.map(lambda ((a,b,c),d):(a,b,c,d))

In [10]:
ratings2= ratingsWithSize2.keyBy(lambda (a,b,c,d):a)

In [12]:
ratings2.take(4)

[(1204723, (1204723, 2048, 0, 379)),
 (1386074, (1386074, 2048, 0, 379)),
 (1060941, (1060941, 2048, 0, 379)),
 (1259031, (1259031, 2048, 0, 379))]

In [13]:
ratingPairs_4s = ratingsWithSize2.keyBy(lambda (a,b,c,d):a).join(ratings2)

In [14]:
ratingPairs_4s.take(4)

[(1179648, ((1179648, 1216, 1, 531), (1179648, 1216, 1, 531))),
 (1114112, ((1114112, 1293, 0, 478), (1114112, 1293, 0, 478))),
 (1114112, ((1114112, 1293, 0, 478), (1114112, 2023, 1, 397))),
 (1114112, ((1114112, 2023, 1, 397), (1114112, 1293, 0, 478)))]

In [15]:
#me quito duplicidades y los que son iguales   
ratingPairs = ratingPairs_4s.map(lambda (x,((a,b,c,d),(a1,b1,c1,d1))):(x,((a,int(b),c,d),(a1,int(b1),c1,d1)))).filter(lambda (x,((a,b,c,d),(a1,b1,c1,d1))):b<b1)

In [16]:
ratingPairs.count()

1333474

In [17]:
ratingPairs.take(3)

[(1114112, ((1114112, 1293, 0, 478), (1114112, 2023, 1, 397))),
 (1073672, ((1073672, 1369, 0, 442), (1073672, 1705, 1, 394))),
 (1073672, ((1073672, 1369, 0, 442), (1073672, 3934, 0, 143)))]

In [18]:
#z el rating , 
vectorial_params = ratingPairs.map(lambda (a,((x,y,z,k),(x1,y1,z1,k1))):((y,y1),(z*z1, z,z1,z*z,z1*z1,k,k1,1)) )

In [58]:
#vectorial_params_2 = vectorial_params.groupByKey().map(lambda (x,iterator):(x,(len(iterator),sum(iterator[1]))))

In [19]:
vectorial_params.take(4)

[((1293, 2023), (0, 0, 1, 0, 1, 478, 397, 1)),
 ((1369, 1705), (0, 0, 1, 0, 1, 442, 394, 1)),
 ((1369, 3934), (0, 0, 0, 0, 0, 442, 143, 1)),
 ((1369, 2567), (0, 0, 0, 0, 0, 442, 302, 1))]

In [56]:
#vectorial_params_2 = vectorial_params.reduceByKey(lambda ((a,b,c,d,e,f,g),(a1,b1,c1,d1,e1,f1,g1)):(a+a1,b+b1,c+c1,d+d1,e+e1,f+f1,g+g1))

In [20]:
vectorial_params_3 = vectorial_params.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1],x[2]+y[2],x[3]+y[3],x[4]+y[4],x[5]+y[5],x[6]+y[6],x[7]+y[7]))

In [21]:
vectorial_params_3.take(5)

[((2372, 3466), (0, 0, 0, 0, 0, 326, 192, 1)),
 ((1675, 2927), (1, 1, 1, 1, 1, 440, 293, 1)),
 ((1377, 1813), (0, 0, 0, 0, 0, 418, 396, 1)),
 ((2991, 4515), (0, 0, 1, 0, 1, 263, 60, 1)),
 ((1484, 2410), (0, 0, 1, 0, 1, 466, 356, 1))]

In [22]:
vectorial_params.count()-vectorial_params_3.count()

176209

In [23]:
def correlation(size, dotProduct, ratingSum,rating2Sum,ratingNormSq,rating2NormSq):
    numerator = size * dotProduct - ratingSum * rating2Sum
    denominator = math.sqrt(size * ratingNormSq - ratingSum * ratingSum)*math.sqrt(size * rating2NormSq - rating2Sum * rating2Sum)
    if denominator == 0:
        corr =0 
    else:
        corr = numerator / denominator
    return  corr

In [24]:
simmil_metrics= vectorial_params_3.mapValues(lambda (a,b,c,d,e,f,g,h):(correlation(h,a,b,c,d,e),h))

#simmil_metrics= vectorial_params_3.map(lambda ((_1,_2),(a,b,c,d,e,f,g,h)):(correlation(h,a,b,c,d,e)))
#,cosineSimilarity(a,math.sqrt(d),math.sqrt(e))


In [25]:
simmil_metrics.sortBy(lambda ((a,b),(c,d)):-c).take(50)

[((1111, 1555), (1.0000000000000002, 4)),
 ((1252, 1778), (1.0000000000000002, 4)),
 ((1272, 1710), (1.0000000000000002, 4)),
 ((1536, 1766), (1.0000000000000002, 4)),
 ((1449, 2661), (1.0000000000000002, 4)),
 ((1172, 1605), (1.0000000000000002, 4)),
 ((1163, 1376), (1.0000000000000002, 4)),
 ((1581, 1582), (1.0000000000000002, 4)),
 ((1536, 1776), (1.0000000000000002, 4)),
 ((1311, 2661), (1.0000000000000002, 4)),
 ((1462, 1834), (1.0000000000000002, 4)),
 ((1212, 1420), (1.0000000000000002, 4)),
 ((1162, 1790), (1.0000000000000002, 4)),
 ((2304, 2440), (1.0000000000000002, 4)),
 ((1562, 2270), (1.0000000000000002, 4)),
 ((1930, 2397), (1.0000000000000002, 4)),
 ((1166, 1393), (1.0000000000000002, 4)),
 ((1599, 2494), (1.0000000000000002, 4)),
 ((1542, 1905), (1.0000000000000002, 4)),
 ((1189, 1644), (1.0000000000000002, 4)),
 ((1487, 1520), (1.0000000000000002, 4)),
 ((1338, 2451), (1.0000000000000002, 4)),
 ((1216, 1237), (1.0000000000000002, 4)),
 ((1236, 1361), (1.000000000000000

In [53]:
#obtengo los items que guardan algún tipo de similaridad
item = 1111
sample = simmil_metrics.filter(lambda ((a,b),(x,y)): a==item or b==item).collect()

In [59]:
sample_sz= len(sample)
simil_sz = simmil_metrics.count()


1157265

In [ ]:
 cosineSimilarity(dotProduct, scala.math.sqrt(ratingNormSq), scala.math.sqrt(rating2NormSq))